<a href="https://colab.research.google.com/github/OmkarNarvekar001/ART_GENERATION_USING_SPEECH_EMOTIONS/blob/main/speech_emotion_detection_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install librosa soundfile numpy sklearn pyaudio

  ERROR: Failed building wheel for pyaudio
  Running setup.py clean for pyaudio
Failed to build pyaudio
    Running setup.py install for pyaudio ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-mbb1fmfa/pyaudio_3590ca0951ff4c8faf98e7e6a000b37e/setup.py'"'"'; __file__='"'"'/tmp/pip-install-mbb1fmfa/pyaudio_3590ca0951ff4c8faf98e7e6a000b37e/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-411105il/install-record.txt --single-version-externally-managed --compile --install-headers /usr/local/include/python3.7/pyaudio Check the logs for full command output.


In [ ]:

import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [ ]:
#DataFlair - Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
    if mel:
        mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        result=np.hstack((result, mel))
    return result

In [ ]:
#DataFlair - Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
#DataFlair - Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.listdir("/content/drive/MyDrive/speech-dataset-2/speech-emotion-recognition-ravdess-data")

['Actor_07',
 'Actor_08',
 'Actor_06',
 'Actor_10',
 'Actor_05',
 'Actor_09',
 'Actor_02',
 'Actor_03',
 'Actor_04',
 'Actor_01',
 'Actor_16',
 'Actor_12',
 'Actor_17',
 'Actor_14',
 'Actor_18',
 'Actor_15',
 'Actor_13',
 'Actor_19',
 'Actor_11',
 'Actor_20',
 'Actor_22',
 'Actor_23',
 'Actor_24',
 'Actor_21']

In [ ]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/speech-dataset-2/speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [ ]:
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [ ]:
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [ ]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [ ]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [ ]:
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [ ]:

y_pred=model.predict(x_test)

In [ ]:
y_pred[-1]

'fearful'

In [ ]:
import pickle
pickle.dump(model, open('/content/drive/MyDrive/model/speech_emotion.sav', 'wb'))

In [ ]:
loaded_model = pickle.load(open('/content/drive/MyDrive/model/speech_emotion.sav', 'rb'))
y_pred=loaded_model.predict(x_test)

In [ ]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 76.56%


In [ ]:
def get_emotion(file_path):
    feature=extract_feature(file_path, mfcc=True, chroma=True, mel=True)
    x_demo=x_test.tolist()
    x_demo.append(feature.tolist())
    x_demo1=np.array(x_demo)
    return model.predict(x_demo1).tolist()[-1]    

In [ ]:
from IPython.display import Audio
file_path="/content/drive/MyDrive/audio-files/my_voice3.wav"
Audio(file_path)

In [ ]:
get_emotion(file_path)

'fearful'

In [ ]:
!pip install SpeechRecognition pydub

     |████████████████████████████████| 32.8 MB 1.3 MB/s 


In [ ]:
import speech_recognition as sr
r = sr.Recognizer()
with sr.AudioFile(file_path) as source:
    # listen for the data (load audio to memory)
    audio_data = r.record(source)
    # recognize (convert from speech to text)
    text = r.recognize_google(audio_data)
    print(text)

rainbow sky
